# Trobant paraules, sintagmes, noms i conceptes

Traduït adaptat de https://github.com/Cristianasp/spacy-notebook, podeu veure el curs original d'spaCy a https://course.spacy.io/chapter1

## Introducció a spaCy

Al centre d'spaCy hi ha l'objecte encarregat del processament. Normalment l'anomenem `nlp`.

Per exemple, per crear un objecte `nlp` per a l'anglès, podem carregar una classe de llengua i utilitzar-la com una funció per analitzar text.

Aquest objecte conté tots els diferents components del pipeline.

També inclou regles específiques de la llengua que s'utilitzen per convertir el text en paraules i signes de puntuació. spaCy admet molts idiomes diferents, disponibles a `spacy.lang`.

In [ ]:
import spacy
from spacy import displacy

# és típic carregar els models d'spacy amb el nom "nlp"
nlp = spacy.load("ca_core_news_sm")
# aquest model inclou les següents peces
nlp.pipe_names

In [ ]:
# Importa la llengua
from spacy.lang.ca import Catalan
from spacy.lang.en import English
from spacy.lang.es import Spanish

# Create l'objecte nlp
nlp = Catalan()

Quan processem un text amb l'objecte `nlp`, spaCy crea un objecte `Doc`.

El `Doc` és el contenidor central de spaCy: emmagatzema el text original i totes les anotacions predites.

El `Doc` es comporta com una seqüència de Python: pots iterar sobre els seus tokens o accedir-hi per índex.

In [ ]:
# processa un text amb nlp
doc = nlp("Hola món")

# Itera sobre els tokens de Doc
for token in doc:
    print(token.text)

<img src="img/token.png" /> 

Els objectes `Token` representen els tokens individuals del document, com paraules o signes de puntuació.

Per obtenir un token concret, podem indexar el document.

Cada token disposa de diversos atributs, com ara `.text`.

In [ ]:
doc = nlp("Hola món!")

# indexa Doc per tenir un sol token
token = doc[1]

print(token, type(token))
print(token.text, type(token.text))

<img src="img/span.png" /> 

Un `Span` és un fragment del document format per un o més tokens. Serveix per sintagmes, n-grames o frases.

És només una vista del `Doc`: no conté dades pròpies.

Podem crear un `Span` utilitzant slicing de Python.

In [ ]:
doc = nlp("Hola món, em dic Pol!")

# Un slice del Doc és un objecte Span
span = doc[1:4]

# Obté el text del span mitjançant l'atribut .text
print(span.text)

Aquí pots veure alguns dels atributs de token disponibles:

"i" és l'índex del token dins del document pare.

"text" retorna el text del token.

"is_alpha", "is_punct" i "like_num" retornen valors booleans que indiquen si el token consisteix en caràcters alfanumèrics, si és puntuació o si s'assembla a un número. Per exemple, un token "10" – u, zero – o la paraula "deu" – D, E, U.

Aquests atributs també s'anomenen atributs lèxics: es refereixen a l'entrada del vocabulari i no depenen del context del token.

In [ ]:
doc = nlp("L'entrada costa 50€")

print("Index:   ", [token.i for token in doc])
print("Text:    ", [token.text for token in doc])

print("is_alpha:", [token.is_alpha for token in doc])
print("is_punct:", [token.is_punct for token in doc])
print("like_num:", [token.like_num for token in doc])

## Primers passos

Comencem i provem spaCy!

Part 1: Català

Importa la classe de català de spacy.lang.ca i crea l'objecte nlp.

Crea un doc i imprimeix el seu text.

In [ ]:
# Importa la classe de llengua catalana
from spacy.lang.ca import Catalan

# Crea l'objecte nlp
nlp = Catalan()

# Processa un text
doc = nlp("Això és una frase.")
print(doc.text)

## Exercici Documents, Spans i Tokens

Quan crides nlp sobre una cadena de text, spaCy primer tokenitza el text i crea un objecte document. En aquest exercici, aprendràs més sobre el Doc, així com les seves vistes Token i Span.

- Importa la classe de llengua anglesa i crea l'objecte nlp.
- Processa el text i instancia un objecte Doc a la variable doc.
- Selecciona el primer token del Doc i imprimeix el seu text.
- Crea un slice del Doc per als tokens "girasols grocs" i "els girasols grocs i les abelles".

In [ ]:
text = "M'agraden el girasols grocs i les abelles."

### Solució

In [ ]:
# Import the English language class and create the nlp object
from spacy.lang.ca import Catalan

nlp = Catalan()
doc = nlp(text)

primer_token = doc[0]
print(primer_token.text)

girasols = doc[3:5]
print(girasols.text)

girasols_abelles = doc[3:8]
print(girasols_abelles.text)

## Atributs lèxics

En aquest exemple, utilitzaràs els objectes Doc i Token de spaCy, i atributs lèxics per trobar percentatges en un text. Buscaràs dos tokens consecutius: un número i un signe de percentatge.

Utilitza l'atribut de token `like_num` per comprovar si un token del doc s'assembla a un número.

Obté el token següent al token actual del document. L'índex del token següent al doc és `token.i + 1`.

Comprova si l'atribut text del token següent és un signe de percentatge "%".

In [ ]:
from spacy.lang.en import English

nlp = English()
doc = nlp("In 1990, more than 60% of people in East Asia were in extreme poverty. " "Now less than 4% are.")

### Solució

In [ ]:
for token in doc:
    # comprova si és un número
    if token.like_num:
        # mira si el següent token és un '%'
        next_token = doc[token.i + 1]
        if next_token.text == "%":
            print("Percentage:", token.text)

### Què passa en català (o castellà)?

In [ ]:
from spacy.lang.ca import Catalan

nlp = Catalan()
[t for t in nlp("6%")]

El tokenitzador no separa el %, així que aquest mètode no ens serveix. De fet, no trobarem un número amb `.like_num`.

Sovint us trobareu amb problemes d'aquest estil. spaCy és un bon punt de partida, però a vegades cal modificar-lo. Per sort, està pensat per això. En general, haureu de buscar per internet la solució. En aquest cas podem afegir un sufix al tokenitzador:

In [ ]:
suffixes = nlp.Defaults.suffixes + ["%"]
suffix_regex = spacy.util.compile_suffix_regex(suffixes)
nlp.tokenizer.suffix_search = suffix_regex.search
[t for t in nlp("6%")]

## Models estadístics

Algunes de les coses més interessants que pots analitzar són específiques del context: per exemple, si una paraula és un verb o si un fragment de text és un nom de persona.

### Què són els models estadístics?

Els models estadístics permeten a spaCy fer prediccions en context. Això normalment inclou etiquetes de categoria gramatical, dependències sintàctiques i entitats amb nom.

Els models s'entrenen amb grans conjunts de dades de textos d'exemple etiquetats.

Es poden actualitzar amb més exemples per afinar les seves prediccions – per exemple, per funcionar millor amb les teves dades específiques.

### Paquets de models

https://spacy.io/usage/models

spaCy proporciona diversos paquets de models pre-entrenats que pots descarregar utilitzant l'ordre "spacy download". Per exemple, el paquet "en_core_web_sm" és un model petit d'anglès que admet totes les capacitats bàsiques i està entrenat amb text web.

El mètode spacy.load carrega un paquet de model pel nom i retorna un objecte nlp.

El paquet proporciona els pesos binaris que permeten a spaCy fer prediccions.

També inclou el vocabulari i meta-informació per indicar a spaCy quina classe de llengua utilitzar i com configurar el pipeline de processament.

<img src="img/models.png" />

In [ ]:
! pip -m spacy download ca_core_news_sm
# ! pip -m spacy download es_core_news_sm
# ! pip -m spacy download en_core_web_sm

### Predicció d'etiquetes de categoria gramatical

Per a cada token del Doc, podem imprimir el text i l'atribut "pos_", l'etiqueta de categoria gramatical predita [part of speech (PoS) en anglès].

A spaCy, els atributs que retornen cadenes de text normalment acaben amb un guió baix; els atributs sense guió baix retornen valors enters.

Aquí, el model predeix correctament "menja" com a verb i "pizza" com a nom.

In [ ]:
# Carrega el model petit de català
nlp = spacy.load("ca_core_news_sm")

# Processa un text
doc = nlp("Ella es menja una pizza enorme")

# Itera sobre els tokens
for token in doc:
    # Imprimeix el text i l'etiqueta de categoria gramatical predita
    print(token.text, token.pos_)

### Predicció de dependències sintàctiques

A més de les etiquetes de categoria gramatical, també podem predir com es relacionen les paraules. Per exemple, si una paraula és el subjecte de la frase o un objecte.

L'atribut "dep_" retorna l'etiqueta de dependència predita.

L'atribut head retorna el token principal sintàctic. També pots pensar-ho com el token pare al qual aquesta paraula està vinculada.

In [ ]:
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

### Esquema d'etiquetes de dependència

<img src="img/dep_example.png" />

Per descriure dependències sintàctiques, spaCy utilitza un esquema d'etiquetes estandarditzat. Aquí hi ha un exemple d'algunes etiquetes comunes:

El pronom "She" és un subjecte nominal vinculat al verb – en aquest cas, a "ate".

El nom "pizza" és un objecte directe vinculat al verb "ate". És menjat pel subjecte, "she".

El determinant "the", també conegut com a article, està vinculat al nom "pizza".

### Predicció d'entitats amb nom

Named Entity Recognition (NER) en anglès.

Les entitats amb nom són "objectes del món real" als quals s'assigna un nom – per exemple, una persona, una organització o un país.

La propietat doc.ents et permet accedir a les entitats amb nom predites pel model.

Retorna un iterador d'objectes Span, així que podem imprimir el text de l'entitat i l'etiqueta de l'entitat utilitzant l'atribut "label_".

En aquest cas, el model està predint correctament "Apple" com a organització, "U.K." com a entitat geopolítica i "$1 billion" com a diners.

In [ ]:
# ! pip -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load("en_core_web_sm")

doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
# Carrega el model petit de català
nlp = spacy.load("ca_core_news_sm")


def mostra_ner(text):
    # Processa un text
    doc = nlp(text)

    # Itera sobre les entitats predites
    for ent in doc.ents:
        # Imprimeix el text de l'entitat i l'etiqueta
        print(ent.text, ent.label_)
    print()


textos = (
    "L'empresa Apple està buscant comprar una empresa a Anglaterra per mil milions d'euros",
    "Com em puc fer soci del Barça?",
    "Som a l'Eixample Clínic, prop de Pl Catalunya.",
)

for text in textos:
    mostra_ner(text)

### Consell: el mètode spacy.explain

Una pregunta ràpida: què significa "ORG"? Què passa amb "NNP" o "dobj"?

En els casos en què necessites saber el significat d'una etiqueta de categoria gramatical o una etiqueta de dependència, pots utilitzar la funció auxiliar spacy.explain.

El mateix funciona per a etiquetes de categoria gramatical i etiquetes de dependència.

In [ ]:
spacy.explain("ORG")

In [ ]:
spacy.explain("PER")

In [ ]:
spacy.explain("NNP")

In [ ]:
spacy.explain("dobj")

In [ ]:
spacy.explain("MISC")

## Predicció d'anotacions lingüístiques

Ara podràs provar un dels paquets de models pre-entrenats de spaCy i veure les seves prediccions en acció. Prova-ho amb el teu propi text! Per esbrinar què significa una etiqueta, pots cridar spacy.explain. Per exemple: spacy.explain('PROPN')

### Part 1

Processa el text amb l'objecte nlp i crea un doc.

Per a cada token, imprimeix el text del token, l'atribut pos_ del token i l'atribut dep_ del token.

In [ ]:
nlp = spacy.load("en_core_web_sm")

text = "It's official: Apple is the first U.S. public company to reach a $1 trillion market value"

# Process the text
doc = nlp(text)

for token in doc:
    # Get the token text, part-of-speech tag and dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # This is for formatting only
    print("{:<12}{:<10}{:<10}".format(token_text, token_pos, token_dep))

In [ ]:
spacy.explain("nsubj")

In [ ]:
nlp = spacy.load("ca_core_news_sm")

text = "És oficial: Apple és la primera empresa americana a assolir el valor de mercat d'un bilió d'euros"

# Processa el text
doc = nlp(text)

for token in doc:
    # Obté el text del token, l'etiqueta de categoria gramatical i l'etiqueta de dependència
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # Això és només per formatar
    print("{:<12}{:<10}{:<10}".format(token_text, token_pos, token_dep))

In [ ]:
spacy.explain("cop")

In [ ]:
spacy.explain("ROOT")

In [ ]:
spacy.explain("acl")

In [ ]:
spacy.explain("nummod")

In [ ]:
spacy.explain("nmod")

In [ ]:
nlp = spacy.load("ca_core_news_sm")

text = "La meva entrada encara no ha arribat. Com faig per entrar al partit"

# Processa el text
doc = nlp(text)

for token in doc:
    # Obté el text del token, l'etiqueta de categoria gramatical i l'etiqueta de dependència
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # Això és només per formatar
    print("{:<12}{:<10}{:<10}".format(token_text, token_pos, token_dep))

In [ ]:
spacy.explain("acl")

### Part 2

Processa el text i crea un objecte doc.

Itera sobre doc.ents i imprimeix el text de l'entitat i l'atribut label_.

In [ ]:
nlp = spacy.load("en_core_web_sm")

text = "It's official: Apple is the first U.S. public company to reach a $1 trillion market value"

# Process the text
doc = nlp(text)

# Iterate over the predicted entities
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

In [ ]:
nlp = spacy.load("en_core_web_sm")

text = "I want to buy a tennis shoes and pay 120 dollars"

# Process the text
doc = nlp(text)

# Iterate over the predicted entities
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

## Predicció d'entitats amb nom en context

Els models són estadístics i no sempre tenen raó. Si les seves prediccions són correctes depèn de les dades d'entrenament i del text que estàs processant. Vegem un exemple.

Processa el text amb l'objecte nlp.

Itera sobre les entitats i imprimeix el text de l'entitat i l'etiqueta.

Sembla que el model no ha predit "iPhone X". Crea un span per a aquests tokens manualment.

In [ ]:
nlp = spacy.load("en_core_web_sm")

text = "New iPhone X release date leaked as Apple reveals pre-orders by mistake"

# Process the text
doc = nlp(text)

# Iterate over the entities
for token in doc.ents:
    # Print the entity text and label
    print(token.text, token.label_)

# Get the span for "iPhone X"
iphone_x = doc[1:3]

# Print the span text
print("Missing entity:", iphone_x.text)

## Concordança basada en regles

Ara veurem el matcher de spaCy, que et permet escriure regles per trobar paraules i frases en text.

Comparat amb expressions regulars, el matcher treballa amb objectes Doc i Token en lloc de només cadenes de text.

També és més flexible: pots cercar textos però també altres atributs lèxics.

Fins i tot pots escriure regles que utilitzin les prediccions del model.

Per exemple, trobar la paraula "duck" només si és un verb, no un nom.

Per què no només expressions regulars?

<ul>
<li>Concordança amb objectes Doc, no només cadenes de text</li>

<li>Concordança amb tokens i atributs de token</li>

<li>Utilitza les prediccions del model</li>

<li>Exemple: "duck" (verb) vs. "duck" (nom)</li></ul>

Els patrons de concordança són llistes de diccionaris. Cada diccionari descriu un token. Les claus són els noms dels atributs de token, mapeats als seus valors esperats.

En aquest exemple, estem buscant dos tokens amb el text "iPhone" i "X".

També podem fer concordances amb altres atributs de token. Aquí, estem buscant dos tokens les etiquetes de categoria gramatical dels quals són "VERB" i "NOUN".

### Ús del Matcher (1)

Per utilitzar un patró, primer importem el matcher de spacy.matcher.

També carregem un model i obtenim l'objecte nlp.

El matcher s'inicialitza amb el vocabulari compartit, nlp.vocab. El necessitaràs més endavant.

El mètode matcher.add et permet afegir un patró. El primer argument és un ID únic per identificar quin patró s'ha trobat. El segon argument és una llista de patrons.

Per fer concordar el patró amb un text, podem cridar el matcher amb qualsevol doc.

Això retornarà les concordances.

In [ ]:
# Import the Matcher
from spacy.matcher import Matcher

# Load a model and create the nlp object
nlp = spacy.load("en_core_web_sm")

# Initialize the matcher with the shared vocab
matcher = Matcher(nlp.vocab)

# Add the pattern to the matcher
pattern = [{"TEXT": "iPhone"}, {"TEXT": "X"}]
matcher.add("IPHONE_PATTERN", [pattern])

# Process some text
doc = nlp("New iPhone X release date leaked")

# Call the matcher on the doc
matches = matcher(doc)

In [ ]:
matches

### Ús del Matcher (2)

Quan cridades el matcher amb un doc, retorna una llista de tuples.

Cada tupla consta de tres valors: l'ID de la concordança, l'índex d'inici i l'índex final del span concordat.

Això significa que podem iterar sobre les concordances i crear un objecte Span: un fragment del doc a l'índex d'inici i final.

In [ ]:
# Call the matcher on the doc
doc = nlp("New iPhone X release date leaked")
matches = matcher(doc)

# Iterate over the matches
for match_id, start, end in matches:
    # Get the matched span
    matched_span = doc[start:end]
    print(matched_span.text)

### Concordança d'atributs de lèxic

Aquí tens un exemple d'un patró més complex utilitzant atributs de lèxic.

Estem buscant cinc tokens:

Un token format només de dígits.

Tres tokens sensibles a majúscules per a "fifa", "world" i "cup".

I un token que consisteix en puntuació.

El patró coincideix amb els tokens "2018 FIFA World Cup:".

In [ ]:
pattern = [
    {"IS_DIGIT": True},
    {"LOWER": "fifa"},
    {"LOWER": "world"},
    {"LOWER": "cup"},
    {"IS_PUNCT": True},
]

doc = nlp("2018 FIFA World Cup: France won!")

### Concordança amb altres atributs de token

En aquest exemple, estem buscant dos tokens:

Un verb amb la forma lematitzada "love", seguit d'un nom.

Aquest patró concordarà amb "loved dogs" i "love cats".

In [ ]:
pattern = [{"LEMMA": "love", "POS": "VERB"}, {"POS": "NOUN"}]

doc = nlp("I loved dogs but now I love cats more.")

# Initialize the matcher with the shared vocab
matcher = Matcher(nlp.vocab)

# Add the pattern to the matcher
matcher.add("PETS", [pattern])

# Process some text

# Call the matcher on the doc
matches = matcher(doc)

# Iterate over the matches
for match_id, start, end in matches:
    # Get the matched span
    matched_span = doc[start:end]
    print(matched_span.text)

In [ ]:
pattern = [{"LEMMA": "love", "POS": "VERB"}, {"POS": "NOUN"}]

doc = nlp("I loved dogs. Now I love cats more.")

# Initialize the matcher with the shared vocab
matcher = Matcher(nlp.vocab)

# Add the pattern to the matcher
matcher.add("PETS", [pattern])

# Process some text

# Call the matcher on the doc
matches = matcher(doc)

# Iterate over the matches
for match_id, start, end in matches:
    # Get the matched span
    matched_span = doc[start:end]
    print(matched_span.text)

### Ús d'operadors i quantificadors (1)

Els operadors i quantificadors et permeten definir quantes vegades ha de concordar un token. Es poden afegir utilitzant la clau "OP".

Aquí, l'operador "?" fa que el token determinant sigui opcional, així que concordarà amb un token amb el lema "buy", un substantiu opcional i un últim substantiu.

In [ ]:
pattern = [{"LEMMA": "buy"}, {"POS": "DET", "OP": "?"}, {"POS": "NOUN"}]

doc = nlp("I bought a smartphone. Now I'm buying apps.")

### Ús d'operadors i quantificadors (2)

"OP" pot tenir un de quatre valors:

Un "!" nega el token, així que ha de concordar 0 vegades.

Un "?" fa que el token sigui opcional, i ha de concordar 0 o 1 vegada.

Un "+" fa que el token concordi 1 o més vegades.

I un "*" fa que el token concordi 0 o més vegades.

## Ús del Matcher

Provem d'utilitzar els mètodes basats en regles de spaCy del nou matcher. Utilitzaràs l'exemple de l'exercici anterior i escriuràs un patró que pugui fer concordar la frase "iPhone X" al text.

Importa el Matcher de spacy.matcher.

Inicialitza'l amb el vocab de l'objecte nlp.

Crea un patró que concordi amb els valors "TEXT" per a dos tokens: "iPhone" i "X".

Utilitza el mètode matcher.add per afegir el patró al matcher.

Crida el matcher amb el doc i emmagatzema el resultat a la variable matches.

Itera sobre les concordances i obté el span concordat des de l'índex d'inici fins a l'índex final.

In [ ]:
# Import the Matcher
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
doc = nlp("New iPhone X release date leaked as Apple reveals pre-orders by mistake")

# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Create a pattern matching two tokens: "iPhone" and "X"
pattern = [{"TEXT": "iPhone"}, {"TEXT": "X"}]

# Add the pattern to the matcher
matcher.add("IPHONE_X_PATTERN", [pattern])

# Use the matcher on the doc
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

## Escrivint patrons de concordança

En aquest exercici, practicaràs escrivint patrons de concordança més complexos utilitzant diferents atributs de token i operadors.

### Part 1

Escriu un patró que només concordi amb mencions de les versions completes d'iOS: "iOS 7", "iOS 11" i "iOS 10".

In [ ]:
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "After making the iOS update you won't notice a radical system-wide "
    "redesign: nothing like the aesthetic upheaval we got with iOS 7. Most of "
    "iOS 11's furniture remains the same as in iOS 10. But you will discover "
    "some tweaks once you delve a little deeper."
)

# Write a pattern for full iOS versions ("iOS 7", "iOS 11", "iOS 10")
pattern = [{"TEXT": "iOS"}, {"IS_DIGIT": True}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("IOS_VERSION_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

### Part 2

Escriu un patró que només concordi amb formes de "download" (tokens amb el lema "download"), seguides d'un nom propi (token amb etiqueta de categoria gramatical "PROPN").

In [ ]:
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "i downloaded Fortnite on my laptop and can't open the game at all. Help? "
    "so when I was downloading Minecraft, I got the Windows version where it "
    "is the '.zip' folder and I used the default program to unpack it... do "
    "I also need to download Winzip?"
)

# Write a pattern that matches a form of "download" plus proper noun
pattern = [{"LEMMA": "download"}, {"POS": "PROPN"}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("DOWNLOAD_THINGS_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

### Part 3

Escriu un patró que concordi amb adjectius ("ADJ") seguits d'un o dos noms (un nom i un nom opcional).

In [ ]:
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "Features of the app include a beautiful design, smart search, automatic " "labels and optional voice responses."
)

# Write a pattern for adjective plus one or two nouns
pattern = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN", "OP": "?"}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("ADJ_NOUN_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)